In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="fa87b981-3938-42ad-8bc6-6970be4e7a96",
    resource_group_name="project",
    workspace_name="589project",
)

In [ ]:
import uuid
from azure.ai.ml.entities import ManagedOnlineEndpoint
from azure.ai.ml.entities import ManagedOnlineDeployment
registered_model_name = "598"

online_endpoint_name = "premiumservice-endpoint-" + str(uuid.uuid4())[:8]
latest_model_version = "1"

try:
    endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)
except Exception as e:

    endpoint = ManagedOnlineEndpoint(
        name=online_endpoint_name,
        description="This is an online endpoint for Carplab Premium Services",
        auth_mode="key",
        tags={
            "Production": "PremiumServices",
        },
    )

    endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

    model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

    blue_deployment = ManagedOnlineDeployment(
        name="blue",
        endpoint_name=online_endpoint_name,
        model=model,
        instance_type="Standard_D2as_v4",
        instance_count=1,
    )
    blue_deployment = ml_client.online_deployments.begin_create_or_update(
        blue_deployment
    ).result()

    endpoint.traffic = {"blue": 100}
    ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [11]:
import pandas as pd
patient_df = pd.read_csv('data/Patient.csv')
transactions_df = pd.read_csv('data/Transactions.csv')
patient_details = {}
transaction ={}
trans = transactions_df[transactions_df['Transaction_ID']==1005].to_dict(orient='records')
for tran in trans:
    for key, val in tran.items():
        transaction[key] = val
pats = patient_df[patient_df['Patient_ID']==transaction['Patient_ID']].to_dict(orient='records')
for pat in pats:
    for key, val in pat.items():
        patient_details[key] = val

In [12]:
transaction

{'Transaction_ID': 1005,
 'Patient_ID': 5,
 'Premieum': 1,
 'age': 62,
 'sex': 0,
 'cp': 0,
 'trestbps': 138,
 'chol': 294,
 'fbs': 1,
 'restecg': 1,
 'thalach': 106,
 'exang': 0,
 'oldpeak': 1.9,
 'slope': 1,
 'ca': 3,
 'thal': 2}

In [13]:
patient_details

{'Patient_ID': 5,
 'Name': 'Gunther G.',
 'DOB': '6/17/82',
 'Sex': 'M',
 'Address': '100th Boulevard, Shoreline, WA.',
 'Phone': 2340000571,
 'Provider': 'Dr. Richard Geller',
 'P_Clinic': 'Heart Hope Clinic',
 'P_Address': '184th St., Kirkland, WA.',
 'P_Phone': 2340000584}

In [35]:
data = {
"input_data": {
    "columns": [
        "age",
        "sex",
        "cp",
        "trestbps",
        "chol",
        "fbs",
        "restecg",
        "thalach",
        "exang",
        "oldpeak",
        "slope",
        "ca",
        "thal"
    ],
    "data": [
        [transaction['age'], transaction['sex'], transaction['cp'], transaction['trestbps'], transaction['chol'], transaction['fbs'], 
         transaction['restecg'], transaction['thalach'], transaction['exang'], transaction['oldpeak'], transaction['slope'], 
         transaction['ca'], transaction['thal']]
    ]
}
    }

In [36]:
data

{'input_data': {'columns': ['age',
   'sex',
   'cp',
   'trestbps',
   'chol',
   'fbs',
   'restecg',
   'thalach',
   'exang',
   'oldpeak',
   'slope',
   'ca',
   'thal'],
  'data': [[52, 1, 0, 125, 212, 0, 1, 168, 0, 1.0, 2, 2, 3]]}}